<a href="https://colab.research.google.com/github/anmyvla-beep/python-ai-Mysttonen-Angelika/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о телесериалах

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных о телесериалах с помощью pandas.

**Данные:**
- `tvserialone.csv` — информация о телесериалах: годы выхода, награды и номинации

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл в pandas DataFrame
3. Очищаем и анализируем структуру данных (сериалы, годы, награды)
4. Проверяем качество данных и готовим их к дальнейшему анализу

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Mysttonen-Angelika"):
    !git clone -q https://github.com/anmyvla-beep/python-ai-Mysttonen-Angelika.git

%cd python-ai-Mysttonen-Angelika

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika")

/content/python-ai-Mysttonen-Angelika
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Mysttonen-Angelika


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [12]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_tv = pd.read_csv("data/tvserialone.csv")

print("✅ Загружено строк в df_cost:", len(df_tv))

✅ Загружено строк в df_cost: 1224


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `tv_series` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно названия сериала.
- Столбцы `tv_seriesLabel`, `awardLabel`, `subjectLabel` содержат читаемые подписи (название сериала, награда, предмет номинации).

В этом шаге мы:
- удалим столбец с URL Wikidata (`tv_series`);
- переименуем `tv_seriesLabel → tv_series`, `awardLabel → award`, `subjectLabel → subject`;
- приведём числовые столбцы (`start_year`, `end_year`) к типу `int`.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URL Wikidata и столбцы вида `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа.

In [13]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (идемпотентная версия)

# Проверяем, нужно ли выполнять очистку: если есть столбец с суффиксом "Label" — данные ещё не обработаны
if "tv_seriesLabel" in df_tv.columns:
    # 1. Удаляем технический столбец с URL Wikidata (игнорируем ошибку, если столбца уже нет)
    df_tv = df_tv.drop(columns=["tv_series"], errors="ignore")

    # 2. Переименовываем столбцы с читаемыми подписями
    df_tv = df_tv.rename(columns={
        "tv_seriesLabel": "tv_series",
        "awardLabel": "award",
        "subjectLabel": "subject"
    })

    # 3. Приводим числовые столбцы к целочисленному типу
    df_tv["start_year"] = pd.to_numeric(
        df_tv["start_year"], errors="coerce"
    ).fillna(0).astype(int)

    df_tv["end_year"] = pd.to_numeric(
        df_tv["end_year"], errors="coerce"
    ).fillna(0).astype(int)

    print("✅ Данные успешно очищены и подготовлены к анализу")
else:
    print("ℹ️ Данные уже очищены ранее — повторная обработка не требуется")

print("\n📋 Структура датасета:")
display(df_tv.head())
print("\n🔍 Текущие столбцы:", list(df_tv.columns))

✅ Данные успешно очищены и подготовлены к анализу

📋 Структура датасета:


,tv_series,start_year,end_year,award,subject
0,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучший телевизионный сериа...,NaN
1,Меня зовут Эрл,2005,2009,премия «Спутник» за лучшую женскую роль в теле...,NaN
2,Меня зовут Эрл,2005,2009,Премия «Спутник» за лучшую женскую роль второг...,NaN
3,Меня зовут Эрл,2005,2009,Премия Гильдии киноактёров США за лучшую женск...,NaN
4,Меня зовут Эрл,2005,2009,Премия Гильдии режиссёров Америки за лучшую ре...,NaN



🔍 Текущие столбцы: ['tv_series', 'start_year', 'end_year', 'award', 'subject']


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по бюджету (`capital_cost`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_cost, "Бюджеты мультфильмов (df_cost)")
show_info(df_genre, "Жанры, страны и продолжительность (df_genre)")

print("\n📈 Статистика по бюджету (capital_cost):")
print(df_cost["capital_cost"].describe())

In [ ]:
# То же самое, но в миллионах долларов (млн $)
stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
print("\n📈 Статистика по бюджету в млн $:")
print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** фильмов, стран и жанров есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу строк);
- **какие жанры самые популярные** (Топ‑10 по числу строк).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`df_genre["country"].value_counts().head()` даёт **Топ‑5 стран по числу записей**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Датасет 1: бюджеты
print("\nУникальных мультфильмов в df_cost:", df_cost["film"].nunique())
print("Диапазон бюджетов (млн $):",
      df_cost["capital_cost"].min() / 1e6, "—", df_cost["capital_cost"].max() / 1e6)

# Датасет 2: жанры, страны, длительность
print("\nУникальных мультфильмов в df_genre:", df_genre["film"].nunique())
print("Уникальных стран:", df_genre["country"].nunique())
print("Уникальных жанров:", df_genre["genre"].nunique())

print("\nТоп-5 стран по числу записей:")
print(df_genre["country"].value_counts().head())

print("\nТоп-10 жанров:")
print(df_genre["genre"].value_counts().head(10))

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨